This experiment is buggy. First of all, it is highly initialization dependant, resulting in 53% to 59% accuracy in consecutive runs. The results are also somehow unnatural.

In [1]:
import csv
import numpy as np
import tensorflow as tf

# Dataset manipulation

## Dataset constants

In [2]:
NUMBER_OF_HEROES = 114
DATASET_SIZE = 500000
TEST_RATIO = 0.25

Converts a list of hero IDs from the heroes.json file to a list of 228 elements, where the 1 value means the hero at that index is in the game

In [3]:
def index_heroes(heroes):
    heroes_indexed = np.zeros(2 * NUMBER_OF_HEROES)

    for i in range(10):
        if i < 5:
            heroes_indexed[int(heroes[i]) - 1] = 1.0
        else:
            heroes_indexed[int(heroes[i]) + 113] = 1.0

    return heroes_indexed

## Transform the file into a list

In [4]:
input_file = open('706d.csv', 'rt')

In [5]:
csv_reader = csv.reader(input_file, delimiter=',')
data_list = list(csv_reader)

print data_list[0]

['3243130796', '1', '85', '58', '104', '106', '1', '87', '108', '109', '33', '69', '1', '3208']


## Split the dataset into training and test sets

In [6]:
train_count = int(DATASET_SIZE * (1 - TEST_RATIO))
test_count = DATASET_SIZE - train_count

x_train = np.zeros((train_count, 2 * NUMBER_OF_HEROES))
y_train = np.zeros(train_count)

x_test = np.zeros((test_count, 2 * NUMBER_OF_HEROES))
y_test = np.zeros(test_count)

# print x_train.shape

## Extract only the relevant information
The features are the heroes in the game as a list of 228 binary elements and the output is 1 if radiant won the game, 0 otherwise.

In [7]:
for i in range(train_count):
    heroes_indexed = index_heroes(data_list[i][2:12])
    x_train[i] = heroes_indexed
    y_train[i] = float(data_list[i][1])

for j in range(test_count):
    heroes_indexed = index_heroes(data_list[train_count + j][2:12])
    x_test[j] = heroes_indexed
    y_test[j] = float(data_list[train_count + j][1])

# Tensorflow operations

In [8]:
learning_rate = 0.001
training_epochs = 20
batch_size = 10000
display_step = 1

In [14]:
n_inputs = 228
n_hidden1 = 1000
n_hidden2 = 1000
n_outputs = 2

In [15]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [16]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [17]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [18]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [19]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [20]:

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [21]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [22]:
hero_dict = {}

In [23]:
n_epochs = 30
batch_size = 500

hero_list = [102, 65, 74, 77, 86, 94, 59, 13, 84]

In [24]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(x_train.shape[0] // batch_size):
            batch_x = x_train[iteration * batch_size:(iteration + 1) * batch_size, :]
            batch_y = y_train[iteration * batch_size:(iteration + 1) * batch_size]
            
            sess.run(training_op, feed_dict={X: batch_x, y: batch_y})
        acc_train = accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        acc_test = accuracy.eval(feed_dict={X: x_test,
                                            y: y_test})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

    hero_dict = {}
    for i in range(114):
        new_list = np.zeros((1, 228))
        
        if (i + 1) not in hero_list and (i + 1) != 24:
            hero_list.append(i + 1)
            
            for j in range(10):
                if j < 5:
                    new_list[0][hero_list[j] - 1] = 1
                else:
                    new_list[0][hero_list[j] - 1 + 114] = 1

            del hero_list[-1]
            
            Z = logits.eval(feed_dict={X: new_list})
            hero_dict[i + 1] = Z[0][0]
        

(0, 'Train accuracy:', 0.59799999, 'Test accuracy:', 0.56466413)
(1, 'Train accuracy:', 0.60999995, 'Test accuracy:', 0.57451218)
(2, 'Train accuracy:', 0.60999995, 'Test accuracy:', 0.57848811)
(3, 'Train accuracy:', 0.61399996, 'Test accuracy:', 0.58072811)
(4, 'Train accuracy:', 0.60800004, 'Test accuracy:', 0.5826081)
(5, 'Train accuracy:', 0.61000001, 'Test accuracy:', 0.58387214)
(6, 'Train accuracy:', 0.61000001, 'Test accuracy:', 0.58521605)
(7, 'Train accuracy:', 0.60999995, 'Test accuracy:', 0.58571208)
(8, 'Train accuracy:', 0.61199999, 'Test accuracy:', 0.58608812)
(9, 'Train accuracy:', 0.61199999, 'Test accuracy:', 0.58653605)
(10, 'Train accuracy:', 0.61400002, 'Test accuracy:', 0.58706415)
(11, 'Train accuracy:', 0.61199999, 'Test accuracy:', 0.58754414)
(12, 'Train accuracy:', 0.61199999, 'Test accuracy:', 0.58764023)
(13, 'Train accuracy:', 0.61399996, 'Test accuracy:', 0.58793616)
(14, 'Train accuracy:', 0.61000001, 'Test accuracy:', 0.58844012)
(15, 'Train accuracy:

In [25]:
import json
json_file = json.load(open("heroes.json", "rt"))
heroes = json_file["heroes"]

def find_hero(index):
    for hero in heroes:
        if int(hero["id"]) == index:
            return hero["localized_name"]

In [26]:
import operator
sorted_dict = sorted(hero_dict.items(), key=operator.itemgetter(1), reverse=True)

for (k, v) in sorted_dict:
    print (find_hero(k) ,v)

(u'Bloodseeker', 0.18943946)
(u'Skeleton King', 0.17385648)
(u'Sven', 0.17081083)
(u'Phantom Assassin', 0.14096205)
(u'Axe', 0.13011174)
(u'Abyssal Underlord', 0.11923712)
(u'Spectre', 0.11279327)
(u'Troll Warlord', 0.077175736)
(u'Crystal Maiden', 0.057774529)
(u'Visage', 0.057242323)
(u'Centaur Warrunner', 0.050076481)
(u'Clockwerk', 0.04910418)
(u'Bane', 0.037684917)
(u'Slardar', 0.0087103015)
(u'Terrorblade', -0.015610469)
(u'Bristleback', -0.023496594)
(u'Zeus', -0.023852326)
(u'Sand King', -0.02429077)
(u'Jakiro', -0.024937388)
(u'Anti-Mage', -0.025404524)
(u'Spirit Breaker', -0.029957987)
(u'Doom', -0.048753727)
(u'Lion', -0.058821943)
(u'Undying', -0.061064679)
(u'Outworld Devourer', -0.061304316)
(u'Viper', -0.066293232)
(u'Phantom Lancer', -0.066336207)
(u'Timbersaw', -0.079281338)
(u'Pugna', -0.081107251)
(u'Windranger', -0.091414429)
(u'Bounty Hunter', -0.091937706)
(u'Gyrocopter', -0.093054041)
(u'Luna', -0.1007828)
(u'Faceless Void', -0.103449)
(u'Winter Wyvern', -0.10882